# DLFeat - CIFAR-10 Classification Example

This notebook demonstrates:
1. Loading the CIFAR-10 image dataset using torchvision.datasets.
2. Extracting features using DLFeat.
3. Training a K-Nearest Neighbors (KNN) classifier on these features.
4. Evaluating the classifier.

Let's start by installing the library:

In [1]:
pip install git+https://github.com/antoninofurnari/dlfeat.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/antoninofurnari/dlfeat.git to /tmp/pip-req-build-55hqdhvi
  Running command git clone --filter=blob:none --quiet https://github.com/antoninofurnari/dlfeat.git /tmp/pip-req-build-55hqdhvi
  Resolved https://github.com/antoninofurnari/dlfeat.git to commit a7ea5182db0a1e045b6ca6023e423a432326a02d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dlfeat: filename=dlfeat-0.6.0-py3-none-any.whl size=16415 sha256=37773fe7009e81b49bb109c5eb60c3efc9b5e294778af52132aeac82597b064c
  Stored in directory: /tmp/pip-ephem-wheel-cache-fuyx2w6t/wheels/e2/c5/ca/a1cbacfdc2a7eb84fce7af09bbe150c8b2d46bfb1d4f4c32d7
Successfully built dlfeat
Note: you may need to restart the kernel to use updated packages.


You'll also need scikit-learn, Pillow, numpy, torch, torchvision for this example. These are generally installed in Colab, but you can install them with the following command if needed:

```
pip install scikit-learn Pillow numpy torch torchvision
```

The code is reported below:

In [2]:
from dlfeat import DLFeatExtractor
from torchvision.datasets import CIFAR10
from torchvision import transforms as tv_transforms # Use a different alias for torchvision transforms
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from PIL import Image
import os

print("--- DLFeat CIFAR-10 & KNN Classification Demo ---")

# --- 1. Load CIFAR-10 Dataset ---
print("\n1. Loading CIFAR-10 dataset...")
# CIFAR-10 images are 32x32. torchvision.datasets returns PIL Images by default.
# We'll download it if not present.
try:
    # Using a transform to get PIL Images directly if not default
    # By default, CIFAR10 returns PIL Image if transform is None or doesn't include ToTensor
    cifar10_train_raw = CIFAR10(root='./cifar_data', train=True, download=True)
    cifar10_test_raw = CIFAR10(root='./cifar_data', train=False, download=True)

    # Extract a subset for faster demonstration (e.g., first 5000 train, 1000 test)
    # Full dataset: 50000 train, 10000 test
    num_train_samples = 5000
    num_test_samples = 1000

    X_train_pil = [cifar10_train_raw[i][0] for i in range(num_train_samples)] # PIL Images
    y_train = [cifar10_train_raw[i][1] for i in range(num_train_samples)]

    X_test_pil = [cifar10_test_raw[i][0] for i in range(num_test_samples)] # PIL Images
    y_test = [cifar10_test_raw[i][1] for i in range(num_test_samples)]

    # Also get raw pixel data for comparison (flattened)
    # To do this, we need to convert PIL to numpy and flatten
    X_train_raw_pixels_flat = np.array([np.array(img).flatten() for img in X_train_pil])
    X_test_raw_pixels_flat = np.array([np.array(img).flatten() for img in X_test_pil])


    print(f"   Subset of dataset loaded: {len(X_train_pil)} training samples, {len(X_test_pil)} testing samples.")
    print(f"   Sample image type: {type(X_train_pil[0])}, mode: {X_train_pil[0].mode}, size: {X_train_pil[0].size}")
    print(f"   Number of classes: {len(cifar10_train_raw.classes)}")

except Exception as e:
    print(f"Error loading or processing CIFAR-10: {e}")
    print("Please ensure you have an internet connection for download and torchvision is working.")
    print("Exiting demo.")
    exit()


# --- 2. Initialize DLFeatExtractor ---
# Using resnet18, which is relatively fast.
MODEL_NAME = "resnet18"
print(f"\n2. Initializing DLFeatExtractor with model: {MODEL_NAME}...")
try:
    feature_extractor = DLFeatExtractor(model_name=MODEL_NAME, task_type="image")
    print(f"   DLFeatExtractor for '{MODEL_NAME}' initialized successfully.")
    print(f"   Feature dimension will be: {feature_extractor.get_feature_dimension()}")
except Exception as e:
    print(f"   Error initializing DLFeatExtractor: {e}")
    print("   Ensure the model is configured in DLFeat and dependencies are met.")
    print("   Exiting demo.")
    exit()

# --- 3. Extract Features ---
# The 'fit' method is for scikit-learn compatibility and typically does nothing here.
print("\n3. Extracting features for training set (this might take a moment)...")
# DLFeatExtractor expects a list of PIL Images or file paths. X_train_pil is already a list of PIL Images.
feature_extractor.fit(X_train_pil)
X_train_features = feature_extractor.transform(X_train_pil)
print(f"   Training features shape: {X_train_features.shape}")

print("   Extracting features for testing set...")
X_test_features = feature_extractor.transform(X_test_pil)
print(f"   Testing features shape: {X_test_features.shape}")

# --- 4. Train KNN Classifier on DLFeat Features ---
print("\n4. Training KNN classifier on DLFeat features...")
knn_classifier_dlfeat = KNeighborsClassifier(n_neighbors=5)
knn_classifier_dlfeat.fit(X_train_features, y_train)
print("   KNN classifier (DLFeat) trained.")

# --- 5. Make Predictions and Evaluate (DLFeat Features) ---
print("\n5. Making predictions and evaluating (DLFeat Features)...")
y_pred_dlfeat = knn_classifier_dlfeat.predict(X_test_features)
accuracy_dlfeat = accuracy_score(y_test, y_pred_dlfeat)
print(f"   Accuracy of KNN on DLFeat ('{MODEL_NAME}') features: {accuracy_dlfeat*100:.2f}%")

# --- 6. For comparison: KNN on Raw Pixel Data ---
print("\n6. Training KNN classifier on raw pixel data (for comparison)...")
knn_classifier_raw = KNeighborsClassifier(n_neighbors=5)
knn_classifier_raw.fit(X_train_raw_pixels_flat, y_train)
print("   KNN classifier (Raw Pixels) trained.")

print("   Making predictions and evaluating (Raw Pixels)...")
y_pred_raw = knn_classifier_raw.predict(X_test_raw_pixels_flat)
accuracy_raw = accuracy_score(y_test, y_pred_raw)
print(f"   Accuracy of KNN on raw pixel data: {accuracy_raw*100:.2f}%")


print("\n--- Demo Finished ---")
print("Note: Features from models like ResNet18 (pre-trained on ImageNet) are general-purpose.")
print("While they can improve performance over raw pixels for many tasks,")
print("fine-tuning the feature extractor or using domain-specific models might yield even better results.")

--- DLFeat CIFAR-10 & KNN Classification Demo ---

1. Loading CIFAR-10 dataset...


100%|██████████| 170M/170M [00:06<00:00, 27.7MB/s]


   Subset of dataset loaded: 5000 training samples, 1000 testing samples.
   Sample image type: <class 'PIL.Image.Image'>, mode: RGB, size: (32, 32)
   Number of classes: 10

2. Initializing DLFeatExtractor with model: resnet18...


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 120MB/s]


   DLFeatExtractor for 'resnet18' initialized successfully.
   Feature dimension will be: 512

3. Extracting features for training set (this might take a moment)...
   Training features shape: (5000, 512)
   Extracting features for testing set...
   Testing features shape: (1000, 512)

4. Training KNN classifier on DLFeat features...
   KNN classifier (DLFeat) trained.

5. Making predictions and evaluating (DLFeat Features)...
   Accuracy of KNN on DLFeat ('resnet18') features: 72.30%

6. Training KNN classifier on raw pixel data (for comparison)...
   KNN classifier (Raw Pixels) trained.
   Making predictions and evaluating (Raw Pixels)...
   Accuracy of KNN on raw pixel data: 26.60%

--- Demo Finished ---
Note: Features from models like ResNet18 (pre-trained on ImageNet) are general-purpose.
While they can improve performance over raw pixels for many tasks,
fine-tuning the feature extractor or using domain-specific models might yield even better results.
